In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data_path = "../../../test/data/adult/adult_test.parquet"

data = pd.read_parquet(data_path)
display(data)
X_test = data.drop('income', axis=1)

In [ ]:
import mlflow

In [ ]:
model_path = "./component_registered_lr_01_1647865968_"

In [ ]:
model_direct = mlflow.pyfunc.load_model(model_path)._model_impl

In [ ]:
model_direct.predict_proba(X_test)

In [ ]:
from deployed_model import DeployedModel

In [ ]:
with DeployedModel(model_path) as dm:
    print("Deployed!")
#    from_dm = dm.predict(X_test)
#    assert np.array_equal(from_dm, model_direct.predict(X_test))

Now try using the wrapper which should provide predict and predict_proba

In [ ]:
from model_wrapper import ModelWrapper

In [ ]:
wrapped_dir = ModelWrapper.wrap_mlflow_model(model_path)

In [ ]:
print(wrapped_dir)

In [ ]:
direct_wrapped = mlflow.pyfunc.load_model(str(wrapped_dir))

In [ ]:
dw_preds = direct_wrapped.predict(X_test)

assert np.array_equal(dw_preds['pred'], model_direct.predict(X_test))
assert np.array_equal(dw_preds['pred_proba'], model_direct.predict_proba(X_test))

Now, _deploy_ the wrapper to its own conda environment:

In [ ]:
with DeployedModel(str(wrapped_dir)) as deployed_wrapped_model:
    assert np.array_equal(deployed_wrapped_model.predict(X_test), model_direct.predict(X_test))
    assert np.array_equal(deployed_wrapped_model.predict_proba(X_test), model_direct.predict_proba(X_test))

In [ ]:
type(wrapped_dir)